In [0]:
from pyspark.sql import functions as F

# Read source tables
sales = spark.table('workspace.silver.crm_sales_details').alias('sales')
cust_dim = spark.table('workspace.gold.dim_customers').alias('cust')
prod_dim = spark.table('workspace.gold.dim_products').alias('prod')

# Join sales with customer dimension
sales_cust = sales.join(
    cust_dim,
    F.col('sales.customer_id') == F.col('cust.gold_customer_id'),
    how='left'
)

# Join with product dimension using std_sls_prd_key and std_prd_key
sales_fact = sales_cust.join(
    prod_dim,
    F.col('sales.std_sls_prd_key') == F.col('prod.std_prd_key'),
    how='left'
)

# Select surrogate keys and sales metrics
fact_result = sales_fact.select(
    F.col('cust.gold_customer_sk'),
    F.col('sales.order_number'),
    F.col('sales.customer_id'),
    F.col('sales.order_date'),
    F.col('sales.ship_date'),
    F.col('sales.due_date'),
    F.col('sales.sales_amount'),
    F.col('sales.quantity'),
    F.col('sales.price')
)

# Write to gold fact table
fact_result.write.format("delta").mode("overwrite").saveAsTable("workspace.gold.fact_sales")

display(fact_result)